In [1]:
%cd ../..
## DATA PREPARATION
import experiment.preprocess as preprocess
import experiment.filtering as filtering
import experiment.prep_llm as prep_llm
from candidates_generation import triple_gen
import importlib
import numpy as np
import pandas as pd

/Users/fieng/Project/KGC_RAG/langchain-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/fieng/Project/KGC_RAG


/Users/fieng/Project/KGC_RAG/langchain-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path ='data/freebase/data_sample.csv'
df = pd.read_csv(path)
evaluation_df, candidates_df, missing_df = preprocess.create_experiment_df(path)
filtred_df = filtering.create_filtred_df(df, evaluation_df, missing_df)

filtred_df = filtred_df.merge(evaluation_df, how='left')
filtred_df_sample = filtering.create_sample(filtred_df,sample_size= 500, true_cand_ratio= 0.5)

/Users/fieng/Project/KGC_RAG/experiment/preprocess.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_cand_df['Missing'] = 1
/Users/fieng/Project/KGC_RAG/experiment/preprocess.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  false_cand_df['Missing'] = 0
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fieng. Use `wandb login --relogin` to force relogin


No cuda devices were available. The model runs on CPU
Training epochs on cpu: 100%|██████████| 50/50 [00:48<00:00,  1.03epoch/s, loss=0.513, prev_loss=0.51] 
Evaluating on cpu:   0%|          | 0.00/50.0 [00:00<?, ?triple/s]WARNING:torch_max_mem.api:Encountered tensors on device_types={'cpu'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).
Evaluating on cpu: 100%|██████████| 50.0/50.0 [00:00<00:00, 179triple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.45s seconds


loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
testing.both.optimistic.adjusted_arithmetic_mean_rank,▁
testing.both.optimistic.adjusted_arithmetic_mean_rank_index,▁
testing.both.optimistic.adjusted_geometric_mean_rank_index,▁
testing.both.optimistic.adjusted_hits_at_k,▁
testing.both.optimistic.adjusted_inverse_harmonic_mean_rank,▁
testing.both.optimistic.arithmetic_mean_rank,▁
testing.both.optimistic.count,▁
testing.both.optimistic.geometric_mean_rank,▁
testing.both.optimistic.harmonic_mean_rank,▁
testing.both.optimistic.hits_at_1,▁


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

In [11]:
template = """
1. Use the following pieces of context to determine if the final triple present correct fact or not.\n
2. Is the triple correct: answer "1" if it is correct and "0" otherwise.\n
3. If you don't know the answer, just say that "-1"\n
4. Start the answer with 'Score:'\n
5. A triple represent a relation between the head entity and the tail entity\n

Here similar triples to help you make a decision:
Context: {context}
Here the triple to evaluate:
Triple: {triple}

Helpful Answer:"""

def prompt_answer(prompt_template:str, **kwargs) -> str:
    # Define llm
    llm = Ollama(model="mistral", temperature=0.1)
    prompt = ChatPromptTemplate.from_template(prompt_template)
    model = llm
    chain = prompt | model
    result = chain.invoke(kwargs)
    return result

context_list =  []
triple_list = []
for item in filtred_df_sample.iterrows():
    context_triple = []
    triple = item[1]
    head = triple['Head']
    relation = triple['Relation']
    tail = triple['Tail']
    triple_sentence = f"Head:{head}\t Relation:{relation}\t Tail:{tail}"
    context_triple.append(df[df['Head'] == head].sample(1))
    context_triple.append(df[df['Relation'] == relation].sample(1))
    context_triple.append(df[df['Tail'] == tail].sample(1))
    context_list.append(context_triple)
    triple_list.append(triple_sentence)

score_list = []
for index, triple in enumerate(triple_list):
    context = context_list[index]
    score = prompt_answer(template, triple=triple, context=context)
    score_list.append(score)
    if index%100 == 0:
        print(f'{index} / {len(triple_list)}')


100 / 500
200 / 500
300 / 500
400 / 500


In [7]:
def clean_score(list_score) -> list[float]:
    list_score = [item.rstrip('.') if item.endswith('.') else item for item in list_score]
    list_score = [0 if item == '' else item for item in list_score]
    score_bin = [1 if item == '1'  else 0 for item in list_score]
    return score_bin


def extract_score(text:str)->str:
    # Function to extract score
    # Find the starting index of "Score: "
    start_index = text.find("Score: ") + len("Score: ")

    # Find the end of the number, which could be marked by a non-digit character
    end_index = start_index
    while end_index < len(text) and (text[end_index].isdigit() or text[end_index] == '.'):
        end_index += 1

    # Extract and return the number using slicing
    return text[start_index:end_index]

In [12]:
new_score_list = [extract_score(score) for score in score_list]
new_score_list = clean_score(new_score_list)
filtred_df_sample['Pred'] = new_score_list

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def compute_score(prediction, ground_truth):
    cm = confusion_matrix(ground_truth, prediction)
    TN = cm[0][0]
    FN = cm[1][0]
    TP = cm[1][1]
    FP = cm[0][1]
    accuracy = (TN + TP) / (TN + TP + FN + FP)
    f1_score = (2 * TP) / (2 * TP + FP + FN)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    return accuracy, f1_score, recall, precision

In [14]:
ground_truth = filtred_df_sample['Missing']
prediction = filtred_df_sample['Pred']
accuracy, f1_score, recall, precision = compute_score(prediction, ground_truth)
print(f'Accuracy {accuracy:.2f}\nPrecision:{precision:.2f}\nRecall:{recall:.2f}\nF1:{f1_score:.2f}')

Accuracy 0.66
Precision:0.71
Recall:0.54
F1:0.61
